### loggerについて  
[loggerの参考URL①](https://www.sejuku.net/blog/23149)  
[loggerの考え方②](https://qiita.com/amedama/items/b856b2f30c2f38665701)  
上記考え方を鑑みて次のセルの書き方に落ち着いた。  

なお、ログのクラス階層については以下のイメージ。
1. 葉：LogRecord
1. 枝：Logger
1. 根：Logging

結論として、loggingでログを書くとグローバル変数を操作することになるので、getLoggerを使って枝のログクラスを各プロセスに割り当てるのが望ましいということ。

In [ ]:
from logging import getLogger, Formatter, StreamHandler, FileHandler, DEBUG

# ログの出力名を設定
logger = getLogger(__name__)

# ログのフォーマットを設定（詳細は次のマークダウンテーブル参考）
fmt = Formatter('%(asctime)s %(name)s %(lineno)d %(levelname)s %(message)s')

# ログのコンソール出力の設定
shandler = StreamHandler()
shandler.setLevel('INFO')
shandler.setFormatter(fmt)

# ログのファイル出力先の設定
fhandler = FileHandler('result_tmp/hoge.log')
fhandler.setLevel(DEBUG)
fhandler.setFormatter(fmt)

# ログレベルの設定
logger.setLevel(DEBUG)
logger.addHandler(shandler)
logger.addHandler(fhandler)
# logger.propagate = False

# 実際に書く際は、以下のように処理中に記載してログを取る。
logger.debug('start')
#df = pd.read_csv('input/train.csv')
logger.debug('end')

### ログのフォーマットの書き方
|フォーマット|役割|
|:-:|:-:|
|%(asctime)s|	実行時刻|
|%(filename)s|	ファイル名|
|%(funcName)s|	行番号|
|%(levelname)s|	ログの定義|
|%(lineno)d|	ログレベル名|
|%(message)s|	ログメッセージ|
|%(module)s|	モジュール名|
|%(name)s|	関数名|
|%(process)d|	プロセスID|
|%(thread)d|	スレッドID|